In [2]:
import pandas as pd
import re
from flatten_json import flatten
import json

collect all raw files into one list

In [3]:
#load csv files and json files that do not require flattening or parsing
data_paths_standard = [
            '/Users/antoninalightfoot/Library/Mobile Documents/com~apple~CloudDocs/Documents/TLU/Data analysis/data/Met/MetObjects.csv',
            '/Users/antoninalightfoot/Library/Mobile Documents/com~apple~CloudDocs/Documents/TLU/Data analysis/data/Reina Sofia/raw_data/reina_sofia3.csv',
            '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/Tate/tate_raw.csv',
            '/Users/antoninalightfoot/Library/Mobile Documents/com~apple~CloudDocs/Documents/TLU/Data analysis/data/Centre Pompidou/merged_file.json',
            '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/Moma/Artworks.csv',
            '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/Whitneymuseum/whitney_data_raw.csv',
            '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/National Gallery (DC)/national_gallery_raw.csv',
 ] 
raw_data = []
for df in data_paths_standard:
    # Step 1: Check if '.json' exists in the input
    if '.json' in df:
        data = pd.read_json(df)
    elif '.csv' in df:
        data = pd.read_csv(df, on_bad_lines='skip', low_memory=False)
    else:
        print("Cannot load " + df)
    raw_data.append(data)
   

dataset_names = [
    'met',
    'reina_sofia',
    'tate',
    'pompidou',
    'moma',
    'whitney',
    'national_gallery'
]

In [4]:
# load datasets that require flattening
data_paths_flattening = [
    '/Users/antoninalightfoot/Library/Mobile Documents/com~apple~CloudDocs/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/dataset.json', # Kiasma]
    '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/SMK/smk_all_da.json',
    '/Users/antoninalightfoot/Library/Mobile Documents/com~apple~CloudDocs/Documents/TLU/Data analysis/data/Kiasma/APIexample-master/dataset.json', #Ateneum
    
]

In [5]:
#flatten the dictionary
flat_data = []
dictionary_data = []
for df in data_paths_flattening:
    with open(df, 'r') as file:
        data = json.load(file)
        dictionary_data.append(data)


In [6]:
for df in dictionary_data:
    flattened_data = [flatten(record) for record in df]

    # Step 3: Convert flattened data to pandas DataFrame
    data = pd.DataFrame(flattened_data)
    flat_data.append(data)

In [7]:
#separate kiasma and ateneum
flat_data[0] = flat_data[0][flat_data[0]['responsibleOrganisation'] == 'Kansallisgalleria / Nykytaiteen museo Kiasma']
flat_data[2] = flat_data[2][flat_data[2]['responsibleOrganisation'] == 'Kansallisgalleria / Ateneumin taidemuseo']

In [46]:
flat_data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58264 entries, 0 to 58263
Columns: 499 entries, objectId to keywords_43_sv
dtypes: float64(118), int64(1), object(380)
memory usage: 221.8+ MB


In [8]:
new_datasets = ['kiasma', 'smk', 'ateneum']
dataset_names.extend(new_datasets)
raw_data.extend(flat_data)

In [9]:
import json
import pandas as pd

# File path to the raw JSON data
data_path = '/Users/antoninalightfoot/Documents/GitHub/museum_personal/data/Queensland/raw_data.json'

# Load the JSON data
with open(data_path, 'r') as file:
    data = json.load(file)

# Extract the column names from the "fields" part of the JSON
columns = [field['id'] for field in data['fields']]

# Extract the records (the data)
records = data['records']

# Create the DataFrame
queensland = pd.DataFrame(records, columns=columns)


In [10]:
dataset_names.append('queensland')
raw_data.append(queensland)

create functions for cleaning fields.
artist name, artwork title, artwork medium, and acquisition source, aquisition year, creation year, artist nationality, artist gender

In [52]:
#prepare queensland for analysis
def assign_dates(row, column_name):
    text = row[column_name]
    
    # Use regular expressions to find patterns for years
    years = re.findall(r'\b\d{4}\b', text)  # Find all 4-digit numbers
    
    # If exactly one year is found, it is the birth year
    if len(years) == 1:
        row['birth_date'] = int(years[0])
        row['death_date'] = None  # No death date if only one year is found
    # If exactly two years are found, assign the first to birth_date and the second to death_date
    elif len(years) >= 2:
        row['birth_date'] = int(years[0])
        row['death_date'] = int(years[1])
    else:
        # If no years or more than two years are found, set both to None
        row['birth_date'] = None
        row['death_date'] = None
    
    return row

raw_data[10]['Artist'] = raw_data[10]['Person'].str.split('\n').str.get(0)
raw_data[10]['Nationality'] = raw_data[10]['Person'].str.split('\n').str.get(-1).str.split('1').str.get(0)
raw_data[10]['birth_date'] = None
raw_data[10]['death_date'] = None
raw_data[10] = raw_data[10].apply(assign_dates, axis=1, column_name='Person')

In [ ]:
#removing duplicate columns created with flattening json
raw_data[7] = raw_data[7].loc[:, ~raw_data[7].columns.str.startswith('keywords')]
raw_data[7] = raw_data[7].loc[:, ~raw_data[7].columns.str.startswith('multimedia')]
raw_data[7]['artist_name'] = raw_data[7]['people_0_firstName'] + " " + raw_data[7]['people_0_familyName']

In [59]:
def create_artist_name(df):
    # List of possible column names for the artist name
    possible_columns = ['artist', 'author', 'artist_name', 'Artist Display Name', 'artist name', 'Artist Display Name', 'Artist', 'artists', 
                        'forwarddisplayname', 'production_0_creator']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [60]:
def create_artwork_title(df):
    # List of possible column names for the artist name
    possible_columns = ['object_title', 'artwork_name', 'Title', 'url', 'name', 'title', 'title_fi', 'titles_0_title']
    
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Title'] = df[col]
            break  # Exit loop once we find the first match
    
    return df

In [61]:
def create_artwork_medium(df):
    # List of possible column names for the artist name
    possible_columns = ['Classification', 'classification', 'classifications_0_en','object_type', 'type_artwork', 'artwork_medium', 'Medium', 'medium',
                        'object_names_1_name', 'PhysicalCategory']
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Medium'] = df[col]
            break  # Exit loop once we find the first match
    return df

In [87]:
def extract_numbers(value):
    # Check if the value is NaN or not a string/number
    if pd.isna(value):  # Check for NaN
        return ""  # Return empty string and 0 count for NaN values
    
    # If it's a string, replace symbols with spaces
    if isinstance(value, str):  # If it's a string
        years = re.findall(r'\b\d{4}\b', value)  # Find all 4-digit numbers
        if years:  # If the list is not empty
            year = years[0]
        else:  # If no years are found, set to a default value
            year = ""
    # If it's an int or float, convert to string and process
    elif isinstance(value, (int, float)):  # If it's a number (int or float)
        value = str(value)  # Convert to string
        years = re.findall(r'\b\d{4}\b', value)  # Try to find 4-digit numbers
        if years:  # If the list is not empty
            year = years[0]
        else:  # If no years are found, set to a default value
            year = ""
      
    return year

    
def clean_acquisition_year(df):
   # List of possible column names for the artist name
    possible_columns = ['acquisition_date', 'credit_line', 'AccessionYear', 'artwork_acquisition', 'year_adquisition', 'acquisitionYear',
                        'DateAcquired', 'accession_number', 'accessionnum', 'inventoryNumber', 'acquisition_date_precision', 'AcquiredDate' ]
    df['Year_acquisition'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Year_acquisition'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df 

use functions for raw datasets and dreate a new list with clean datasets

In [ ]:
#I need to add more tags based on dataset values
def classify_medium(df):
    df['Medium_classified'] = pd.NA
    medium_tags = [['paper', 'watercolor', 'card', 'watercolour', 'board', 'chalk', 'mixed', 'print', 'parchment', 'graphic',
                    'engraving', 'etching', 'dessin'], #graphics
                   ['oil', 'canvas', 'paint', 'fresco', 'painting', 'peinture'], #painting
                   ['bronze', 'granite', 'marble', 'alabaster', 'clay', 'iron', 'cement', 'concrete', 'plaster', 'sculpture'], #sculpture
                   ['film', 'video'], #video art
                   ['fabric', 'glass', 'wood', 'aluminium', 'steel', 'plastic', 'book', 'copper', 'stone', 
                    'slate', 'wool', 'willow', 'cedar', 'wax', 'walnut', 'tin', 'plate', 'textile', 'terracota', 
                    'mahogany', 'pine', 'oak', 'iron', 'suede', 'brass', 'sheet', 'cast', 'acacia', 'metal', 'beech', 'celluloid',
                    'cellulose', 'ceramic', 'rubber', 'elm', 'earthenware', 'epoxy', 'felt', 'firebricks', 'flint', 'banknote',
                    'ivory', 'lead', 'leather', 'terracota', 'perspex', 'resin', 'quartz', 'porcelain', 'object', 'medal',  'silk', 'cotton', 'cashmere'], #object
                   ['boat', 'structure', 'burlap', 'bean', 'hook', 'knive', 'hat', 'light', 'wall', 'machine', 
                    'vinyl', 'cleaner', 'door', 'mirror', 'form', 'metronome', 'sock', 'carpet', 'latex', 'installation'], #installation
                   ['photo'], #photography
                   ['software', 'audio', 'slide', 'digital', 'net', 'sound'], #new media
                   ['architecture'], #architecture
                   ]
    medium_name = ['graphics', 'painting', 'sculpture', 'video art', 'object', 'installation', 'photography', 'new media', 'architecture']
     # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        medium = row['Medium']  # Assuming the column is named 'Medium'
        
        # Skip if the 'Medium' value is not a string (e.g., NaN or other types)
        if not isinstance(medium, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(medium_tags):
            # Check if any keyword from the tag group exists in the 'Medium' column
            if any(tag.lower() in medium.lower() for tag in tag_group):
                # Assign the corresponding medium name (from medium_name list)
                df.at[idx, 'Medium_classified'] = medium_name[i]
                break  # Stop once we find a match
    
    return df

In [127]:
raw_data[4]['nationality_raw'].unique()

array(['austrian', 'french', '', ..., 'filipino american', ' thai',
       'hunkpapa lakota'], dtype=object)

In [ ]:
#i need to update the dictionary so all nationalities will be translated into countries
def create_artist_nationality(df):
    possible_columns = ['Country','Artist Nationality', 'nationality_artist', 'artist_nationality', 'Nationality', 'nationality', 
                        'production_0_creator_nationality']
    df['Country_calculated'] = pd.NA
    df['nationality_raw'] = pd.NA

    # Dictionary to map nationalities to countries
    nationality_to_country = {
    'american': 'United States',
    'américaine': 'United States',
    'allemande': 'Germany',
    'canadian': 'Canada',
    'canadienne': 'Canada',
    'japanese': 'Japan',
    'japonaise': 'Japan',
    'mexican': 'Mexico',
    'mexicaine': 'Mexico',
    'indian': 'India',
    'française': 'France',
    'russe': 'Russia',
    'suisse': 'Switzerland',
    'britannique': 'United Kingdom',
    'hongroise': 'Hungary',
    'argentine': 'Argentina',
    'polonaise': 'Poland',
    'yougoslave': 'Yugoslavia',
    'suédoise': 'Sweden',
    'soviétique': 'Soviet Union',
    'italienne': 'Italy',
    'espagnole': 'Spain',
    'brésilienne': 'Brazil',
    'égyptienne': 'Egypt',
    'vénézuélienne': 'Venezuela',
    'israélienne': 'Israel',
    'cubaine': 'Cuba',
    'iranienne': 'Iran',
    'algérienne': 'Algeria',
    'ukrainienne': 'Ukraine',
    'roumaine': 'Romania',
    'belge': 'Belgium',
    'néerlandaise': 'Netherlands',
    'apatride': 'Stateless',
    'autrichienne': 'Austria',
    'irlandaise': 'Ireland',
    'portugaise': 'Portugal',
    'jordanienne': 'Jordan',
    'australienne': 'Australia',
    'croate': 'Croatia',
    'tchèque': 'Czech Republic',
    'albanaise': 'Albania',
    'sud-africaine': 'South Africa',
    'irakienne': 'Iraq',
    'libanaise': 'Lebanon',
    'estonienne': 'Estonia',
    'kosovare': 'Kosovo',
    'marocaine': 'Morocco',
    'ivoirienne': 'Ivory Coast',
    'arménienne': 'Armenia',
    'chinoise': 'China',
    'vietnamienne': 'Vietnam',
    'tchécoslovaque': 'Czechoslovakia'
}


    # Check each possible column for nationality information
    for col in possible_columns:
        if col in df.columns:
            df['nationality_raw'] = df[col].str.replace('(', '').str.replace(')', '').str.lower()  # Copy the nationality info into 'nationality_raw'
            break  # Exit loop once we find the first match 

    # Replace nationalities in 'nationality_raw' with country names
    df['Country_calculated'] = df['nationality_raw'].replace(nationality_to_country)

    return df


In [ ]:

def create_artist_gender(df):
    possible_columns = ['Artist Gender', 'gender', 'artist_gender', 'Gender', 'production_0_creator_gender']

In [ ]:
def create_acquisition_method(df):
    possible_columns = ['Credit Line', 'creditLine', 'acquisition_type', 'CreditLine', 'credit_line', 'creditline', 'CreditLine']

In [75]:
def artwork_creation_date(df):
    possible_columns = ['Object Date', 'year_production', 'year', 'object_date', 'Date', 'display_date', 'endyear_x', 'yearFrom', 
                        'production_date_0_end', 'DateCreated']
    
    df['Date_creation_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Date_creation_year'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df 

In [90]:
def artist_birth_year(df):
    possible_columns = ['Artist Begin Date', 'author_born_year', 'yearOfBirth', 'artist_birth', 'BeginDate', 'birth_date', 
                        'beginyear', 'people_0_birthDate', 'production_0_creator_date_of_birth']
    df['Artist_birth_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_birth_year'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df 
    

In [92]:
def artist_death_year(df):
    possible_columns = ['Artist End Date', 'author_death_year', 'yearOfDeath', 'artist_death', 'EndDate', 'death_date', 'endyear', 
                        'people_0_deathDate', 'production_0_creator_date_of_death']
    df['Artist_death_year'] = pd.NA
  
    # Loop over the possible column names and use the first one that exists
    for col in possible_columns:
        if col in df.columns:
            df['Artist_death_year'] = df[col].apply(extract_numbers)
            break  # Exit loop once we find the first match
    
    return df

In [ ]:
transformed_data = []
for df in raw_data:
    #create_artist_name(df)
    #create_artwork_title(df)
    #create_artwork_medium(df)
    #clean_acquisition_year(df)
    #classify_medium(df)
    #create_artist_nationality(df)
    #artwork_creation_date(df)
    #artist_birth_year(df)
    #artist_death_year(df)
    transformed_data.append(df)


In [128]:
test = transformed_data[4]

save clean datasets in separate csv files

In [ ]:
#i need to add more colums to the final dataset
clean_data = []
for df in transformed_data:
    df = df[[ 'Artist', 'Title','Medium', 'Year_acquisition', 'Medium_classified']]
    clean_data.append(df)

In [ ]:

# Save each dataset using the corresponding name
for dataset, name in zip(clean_data, dataset_names):
    # Generate a unique filename using the dataset name
    file_name = f"{name}.csv"
    
    # Save the DataFrame to a CSV file
    dataset.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved met_clean.csv
Saved reina_sofia_clean.csv
Saved tate_clean.csv
